# First we import important liberaries

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import mysql.connector as mc

db = mc.connect(host = "localhost",
                username = "root",
                password = "Nikunj@24",
                database = "ecommerce")
cur = db.cursor()

# List all unique cities where customers are loacted.

In [4]:
query = """ select distinct customer_city from customers """
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data)
df.head()

,0
0,franca
1,sao bernardo do campo
2,sao paulo
3,mogi das cruzes
4,campinas


# Count the number of orders placed in 2017.

In [12]:
query = "select count(order_purchase_timestamp) as number_of_order from orders where year(order_purchase_timestamp) = 2017"
cur.execute(query)
data = cur.fetchall()
# data[0][0]
"number of order in 2017 = ",data[0][0]

('number of order in 2017 = ', 45101)

# Find the total sales per category

In [19]:
query = """
SELECT 
    p.product_category AS category,
    round(SUM(pay.payment_value),3) AS sales
FROM products AS p
JOIN order_items AS oi
    ON p.product_id = oi.product_id
JOIN payments AS pay
    ON pay.order_id = oi.order_id
GROUP BY p.product_category
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["Category","Total sales"])
df

,Category,Total sales
0,perfumery,506738.660
1,Furniture Decoration,1430176.392
2,telephony,486882.051
3,bed table bath,1712553.671
4,automotive,852294.331
...,...,...
69,cds music dvds,1199.430
70,La Cuisine,2913.530
71,Fashion Children's Clothing,785.670
72,PC Gamer,2174.430


# calculate the percentage of orders that were paid in installments

In [17]:
query = "select sum(case when payment_installments > 0 then 1 else 0 end)/count(*)*100 from payments"
cur.execute(query)
data = cur.fetchall()
"The percentage of orders that where paid in the installments is = ",data[0][0]

('The percentage of orders that where paid in the installments is = ',
 Decimal('99.9981'))

# Count the number of customers from each state. 

In [2]:
query = """select customer_state, count(customer_id)
from customers 
group by customer_state"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["customer_state","customer_id"])
df.head() # give some only 5, for all call only df

,customer_state,customer_id
0,SP,83492
1,SC,7274
2,MG,23270
3,PR,10090
4,RJ,25704


# Calculate the number of orders per month in 2018

In [39]:
query = """select monthname(order_purchase_timestamp) months ,count(order_id) order_count
from orders where year(order_purchase_timestamp) = 2018
group by  months"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["months","order_count"])
df

,months,order_count
0,July,6292
1,August,6512
2,February,6728
3,June,6167
4,March,7211
5,January,7269
6,May,6873
7,April,6939
8,September,16
9,October,4


#  Find the average number of products per order, grouped by customer city

In [36]:
query = """
with count_per_order as
(select o.order_id,o.customer_id,COUNT(oi.order_item_id) as item_count
from orders o
join order_items oi
on o.order_id = oi.order_id
group by o.order_id,o.customer_id)

SELECT 
    customers.customer_city,
    round(AVG(count_per_order.item_count),2) AS Avg_items_per_order
FROM customers
JOIN count_per_order
    ON customers.customer_id = count_per_order.customer_id
GROUP BY customers.customer_city order by Avg_items_per_order desc;
"""

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["customer_city","Avg_item_per_order"])
df.head(10)

,customer_city,Avg_item_per_order
0,padre carvalho,7.00
1,celso ramos,6.50
2,datas,6.00
3,candido godoi,6.00
4,matias olimpio,5.00
5,cidelandia,4.00
6,picarra,4.00
7,morro de sao paulo,4.00
8,teixeira soares,4.00
9,curralinho,4.00


## Calculate the percentage of total revenue contributed by each product category.

In [4]:
query = """
SELECT 
    p.product_category AS category,
    round ((round(SUM(pay.payment_value),3)/(select round(sum(payment_value),3) from payments)) * 100,3) AS sales
FROM products AS p
JOIN order_items AS oi
    ON p.product_id = oi.product_id
JOIN payments AS pay
    ON pay.order_id = oi.order_id
GROUP BY p.product_category order by sales desc
;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["Product Category" , "% of Total Revenue"] )
df



,Product Category,% of Total Revenue
0,bed table bath,10.698
1,HEALTH BEAUTY,10.353
2,computer accessories,9.903
3,Furniture Decoration,8.934
4,Watches present,8.928
...,...,...
69,PC Gamer,0.014
70,House Comfort 2,0.011
71,cds music dvds,0.007
72,Fashion Children's Clothing,0.005


### Identify the correlation between product price and the number of times a product has been purchased.

In [8]:
query = """
select p.product_category,count(oi.product_id),round(avg(oi.price),2) as Avg_price
from products p join order_items oi
on p.product_id = oi.product_id
group by p.product_category;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["Product Category" , "Count" ,"Avg price"] )
df
arr1 = df["Count"]
arr2 = df["Avg price"]

a = np.corrcoef([arr1,arr2])
print("the correlation is", a[0][-1])


the correlation is -0.10631514167157562


# Calculate the total revenue generated by each seller, and rank them by revenue

In [11]:
query = """
select *,dense_rank() over(order by revenue desc) as rk from 
 (select oi.seller_id,sum(pay.payment_value) as revenue
 from order_items oi join payments pay
 on oi.order_id = pay.order_id
 group by oi.seller_id) as a;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["seller_id","revenue","rank"])
df

,seller_id,revenue,rank
0,7c67e1448b00f6e969d365cea6b010ab,507166.907302,1
1,1025f0e2d44d7041d6cf58b6550e0bfa,308222.039840,2
2,4a3ca9315b744ce9f8e9374361493884,301245.269765,3
3,1f50f920176fa81dab994f9023523100,290253.420128,4
4,53243585a1d6dc2643021fd1853d8905,284903.080498,5
...,...,...,...
3090,ad14615bdd492b01b0d97922e87cb87f,19.209999,3082
3091,702835e4b785b67a084280efca355756,18.559999,3083
3092,4965a7002cca77301c82d3f91b82e1a9,16.360001,3084
3093,77128dec4bec4878c37ab7d6169d6f26,15.220000,3085
